In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import string

## 1. Load the dataset

In [3]:
# load datasets given
df_movie_details = pd.read_json("../data/IMDB_movie_details.json", lines = True)
df_reviews = pd.read_json("../data/IMDB_reviews.json", lines = True)

In [5]:
df_reviews = pd.read_pickle("../data/tokenized_reviews.pkl.gz", compression = 'gzip')

## 2. Text Preprocessing

### Tokenize review texts

In [3]:
# TOKENIZING WILL TAKE ~15 MINUTES
# If you want to save after tokenizing feel free to do so to save time in tokenizing again
df_reviews['tokenized_summary'] = list(map(word_tokenize, df_reviews['review_summary']))
df_reviews['tokenized_reviews'] = list(map(word_tokenize, df_reviews['review_text']))

### Removing stop words and punctuations from the list

In [6]:
# get stop words
stop_words_and_punctuations = set(stopwords.words('english') + list(string.punctuation))

In [7]:
# remove stop words and punctuations from the tokenized list
df_reviews['tokenized_summary'] = list(map(lambda x: [word.lower() for word in x if word.lower() not in stop_words_and_punctuations], df_reviews['tokenized_summary']))
df_reviews['tokenized_reviews'] = list(map(lambda x: [word.lower() for word in x if word.lower() not in stop_words_and_punctuations], df_reviews['tokenized_reviews']))

### Stemming or Lemmatisation -- (to be implemented)

In [ ]:
# stem or lemmatise words

### Transform texts to numbers

### Save the new dataset

In [ ]:
# save changes made to original dataset to save time tokenizing etc
df_reviews.to_pickle("../data/cleaned_reviews.pkl.gz", compression = 'gzip')

## 3. Model Building

In [ ]:
# Load the previously saved dataset
df_reviews = pd.read_pickle("../data/cleaned_reviews.pkl.gz", compression = 'gzip')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_reviews.loc[:, df_reviews.columns != 'is_spoiler'], df_reviews.loc[:, 'is_spoiler'], test_size = 0.2)

In [ ]:
logistic_model = LogisticRegression(max_iter = 1e3)
# To be implemented
# logistic_model.fit(X_train['tokenized_reviews'], y_train)

In [ ]:
logistic_model.score(X_test, y_test)